https://ui.adsabs.harvard.edu/abs/2019A%26A...621A.122B/abstract

In [1]:
from astropy import units as u
distance = 2.2*u.kpc

In [2]:
cd /orange/adamginsburg/salt/beuther/

/orange/adamginsburg/salt/beuther


In [3]:
ls

core.python-8-1653072238-3663-3389-c0711a-s14.ufhpc.56877
core.python-8-1653072280-3663-3389-c0711a-s14.ufhpc.21418
disk1.reg
disk2.reg
G351_spw0_selfcal2_disk1.fits
G351_spw0_selfcal2_disk2.fits
G351_spw0_selfcal2.fits
G351_spw1_selfcal2_disk1.fits
G351_spw1_selfcal2_disk2.fits
G351_spw1_selfcal2.fits
G351_spw2_selfcal2_disk1.fits
G351_spw2_selfcal2_disk2.fits
G351_spw2_selfcal2.fits
G351_spw3_selfcal2_disk1.fits
G351_spw3_selfcal2_disk2.fits
G351_spw3_selfcal2.fits
NaCl_v_le_3_stackcube_disk1.fits
NaCl_v_le_3_stackcube_disk2.fits
spectra/
water_masked_moment0.fits


for fn in glob.glob("*.fits"):
    print('app

In [ ]:
%matplotlib inline
import pylab as pl
pl.rcParams['figure.facecolor'] = 'w'
from spectral_cube import SpectralCube
from astropy import units as u
from astropy.visualization import quantity_support
quantity_support()

In [ ]:
from astropy.table import Table
from astropy import table

In [ ]:
import pprint

In [ ]:
import scipy, scipy.ndimage

In [ ]:
import numpy as np
import os
import glob
from astropy import constants, units as u, table, stats, coordinates, wcs, log, coordinates as coord, visualization
import radio_beam
import pyspeckit
import pylab as pl
from astroquery.splatalogue.utils import minimize_table as mt
import sys
sys.path.append('/orange/adamginsburg/salt/Orion_ALMA_2016.1.00165.S/analysis')
import lines
import paths
from salt_tables import (salt_tables, salt_table_names, SO, SO2, HCl, sis_tables, AlCl, AlF, Al37Cl,
                         NaF, AlO, AlOH, NaCN, CaS, CaO)

In [ ]:
import regions
from astropy import units as u, coordinates
reg = regions.Regions.read('../sanhueza/G351.77/cutout.reg')
disk1 = regions.Regions.read('disk1.reg')
disk2 = regions.Regions.read('disk2.reg')
#regvsm = regions.Regions.read('verysmall.reg')
#regvvsm = regions.Regions.read('innerdiskbox.reg')

In [ ]:
cont = SpectralCube.read('../sanhueza/G351.77/G351.77-0.54.config8.cont_avg.selfcal_0.5_hogbom.image.fits')
pl.figure(figsize=(10,10))
ax = pl.subplot(projection=cont[0].wcs)
ax.imshow(cont[0].value, cmap='gray_r', norm=visualization.simple_norm(cont[0].value, stretch='log'))
reg[0].to_pixel(cont[0].wcs).plot(ax=ax)
disk1[0].to_pixel(cont[0].wcs).plot(ax=ax)
disk2[0].to_pixel(cont[0].wcs).plot(ax=ax)
pp = reg[0].to_pixel(cont.wcs.celestial)
bb = pp.bounding_box
ax.axis([bb.ixmin-20, bb.ixmax+20, bb.iymin-20, bb.iymax+20])

# NaCl Stackl

In [ ]:
from spectral_cube.analysis_utilities import stack_cube

In [ ]:
nacltables = salt_tables[3:5]
#nacltables[0]

In [ ]:
import spectral_cube.analysis_utilities
import imp
imp.reload(spectral_cube.analysis_utilities)
from spectral_cube.analysis_utilities import stack_cube

In [ ]:
flist = (glob.glob('G351_spw[0123]_selfcal2.fits'))

In [ ]:
flist

In [ ]:
#from dask.diagnostics import ProgressBar
#pbar = ProgressBar()
#pbar.register()

import dask
dask.config.set(scheduler='threads', num_workers=8)

In [ ]:
for fn in flist:
    for rr,regn in ((disk1, 'disk1'), (disk2, 'disk2')):
        outfile = fn.replace(".fits",f"_{regn}.fits")
        if not os.path.exists(outfile) or (os.path.exists(outfile) and (SpectralCube.read(outfile).unit != u.K) or hasattr(SpectralCube.read(outfile), 'beams')):
            cube = SpectralCube.read(fn, use_dask=True).subcube_from_regions(rr)
            cube.allow_huge_operations=True
            if hasattr(cube, 'beams'):
                with cube.use_dask_scheduler('threads', num_workers=8):
                    cb = cube.beams.common_beam(max_iter=20, max_epsilon=0.01)
                    scube = cube.convolve_to(cb, save_to_tmp_dir=True)
            else:
                scube = cube
            assert not hasattr(scube, 'beams')
            scube.allow_huge_operations = True
            scube.to(u.K).write(outfile, overwrite=True)            

In [ ]:
regn = "disk1"
cubes = [SpectralCube.read(fn, use_dask=True)#.subcube_from_regions(reg3)
         for fn in (glob.glob(f'G351_spw[0123]_selfcal2_{regn}.fits'))]
cubes

In [ ]:
regn = "disk2"
cubesd2 = [SpectralCube.read(fn, use_dask=True)#.subcube_from_regions(reg3)
         for fn in (glob.glob(f'G351_spw[0123]_selfcal2_{regn}.fits'))]
cubesd2

In [ ]:
# only needed for VRSCs
if hasattr(cubes[0], 'beams'):
    beams = radio_beam.Beams(beams=[cube.beams.common_beam(max_iter=20, max_epsilon=0.01)
                                    if hasattr(cube, 'beams')
                                    else cube.beam
                                    for cube in cubes])
    cb=beams.common_beam()
else:
    beams = radio_beam.Beams(beams=[cube.beam for cube in cubes])
    cb=beams.common_beam()

In [ ]:
cb

In [ ]:
(cb.major*distance).to(u.au, u.dimensionless_angles())

In [ ]:
(cb.minor*distance).to(u.au, u.dimensionless_angles())

In [ ]:
linelist = np.concatenate([data['Freq'][(data['vu'] <= 3) &
                                          np.any([((data['Freq'].quantity > cube.with_spectral_unit(u.GHz).spectral_extrema[0]) &
                                                   (data['Freq'].quantity < cube.with_spectral_unit(u.GHz).spectral_extrema[1]))
                                                 for cube in cubes],
                                                 axis=0)].quantity
                             for data in nacltables])
linenames = np.concatenate( [data['Species'][(data['vu'] <= 3) &
                                          np.any([((data['Freq'].quantity > cube.with_spectral_unit(u.GHz).spectral_extrema[0]) &
                                                   (data['Freq'].quantity < cube.with_spectral_unit(u.GHz).spectral_extrema[1]))
                                                 for cube in cubes],
                                                 axis=0)]
                             for data in nacltables] )
(linelist, linenames)

In [ ]:
from astropy import log
log.setLevel('INFO')

In [ ]:
#log.setLevel('DEBUG')
vcen = -3.6 * u.km/u.s # from Patricio
stack, cutouts = stack_cube(cubes,
                   linelist=linelist,
                   vmin=vcen-25*u.km/u.s,
                   vmax=vcen+25*u.km/u.s,
                   average=np.nanmean,
                   convolve_beam=cb,
                   return_cutouts=True)
#log.setLevel('INFO')
stack.to(u.K).write('NaCl_v_le_3_stackcube_disk1.fits', overwrite=True)

In [ ]:
stackc = SpectralCube.read('NaCl_v_le_3_stackcube_disk1.fits')
stackc

In [ ]:
#log.setLevel('DEBUG')
vcen = -3.6 * u.km/u.s # from Patricio
stack2, cutouts2 = stack_cube(cubesd2,
                   linelist=linelist,
                   vmin=vcen-25*u.km/u.s,
                   vmax=vcen+25*u.km/u.s,
                   average=np.nanmean,
                   convolve_beam=cb,
                   return_cutouts=True)
#log.setLevel('INFO')
stack2.to(u.K).write('NaCl_v_le_3_stackcube_disk2.fits', overwrite=True)

In [ ]:
# try creating a vmap based on another line (water)
restval = 232.6867*u.GHz # 'H2Ov2=1_5(5,0)-6(4,3)': 
cube = [c for c in cubes if (c.with_spectral_unit(u.GHz).spectral_extrema[0] < restval) & (c.with_spectral_unit(u.GHz).spectral_extrema[1] > restval)][0]
print(cube)
scube = (cube.subcube_from_regions(disk1)
         .with_spectral_unit(u.GHz)
         .with_spectral_unit(u.km/u.s,
                             velocity_convention='radio',
                             rest_value=restval)
         .spectral_slab(-20*u.km/u.s+vcen, 20*u.km/u.s+vcen))
print(scube)
pl.figure(figsize=(14,5))
pl.subplot(1,3,1)
m0 = scube.moment0()
m0.quicklook()
pl.title("m0")

pl.subplot(1,3,2)
mx = scube.max(axis=0)
mx.quicklook()
pl.title("mx")
pl.colorbar()
pl.subplot(1,3,3)
msk = mx > 150*u.K
msk = scipy.ndimage.binary_erosion(msk, iterations=1)
msk = scipy.ndimage.binary_dilation(msk, iterations=7)
pl.imshow(msk, interpolation='none', origin='lower')
pl.title('mask')
pl.tight_layout()
pl.figure()

from dask_image.ndmorph import binary_dilation, binary_erosion
m3d = (scube > 40*u.K).include()
m3d = binary_erosion(m3d, iterations=1)
m3d = binary_dilation(m3d, iterations=2)
m3d = m3d.compute()

m1 = scube.with_mask(msk).with_mask(m3d).moment1()
m0m = scube.with_mask(msk).with_mask(m3d).moment0()
pl.figure(figsize=(14,6))
ax1 = pl.subplot(1,2,1)
im = ax1.imshow(m1.value, vmin=vcen.value-10, vmax=vcen.value+10)
pl.colorbar(mappable=im)
mxv = scube.with_mask(msk).argmax_world(axis=0)
pl.subplot(1,2,2)
pl.imshow(mxv.value, vmin=vcen.value-10, vmax=vcen.value+10)
pl.colorbar()

pl.figure()
pl.imshow(m0m.value)
m0m.hdu.writeto('water_masked_moment0.fits', overwrite=True)

vmap = m1.hdu

In [ ]:
# try creating a vmap based on another line (water)
restval = 232.6867*u.GHz # 'H2Ov2=1_5(5,0)-6(4,3)': 
vcen2 = -6*u.km/u.s
cube = [c for c in cubesd2 if (c.with_spectral_unit(u.GHz).spectral_extrema[0] < restval) & (c.with_spectral_unit(u.GHz).spectral_extrema[1] > restval)][0]
print(cube)
scube = (cube.subcube_from_regions(disk2)
         .with_spectral_unit(u.GHz)
         .with_spectral_unit(u.km/u.s,
                             velocity_convention='radio',
                             rest_value=restval)
         .spectral_slab(-20*u.km/u.s+vcen2, 20*u.km/u.s+vcen2))
print(scube)
pl.figure(figsize=(14,5))
pl.subplot(1,3,1)
m0 = scube.moment0()
m0.quicklook()
pl.title("m0")

pl.subplot(1,3,2)
mx = scube.max(axis=0)
mx.quicklook()
pl.title("mx")
pl.colorbar()
pl.subplot(1,3,3)
msk = mx > 50*u.K
msk = scipy.ndimage.binary_erosion(msk, iterations=2)
msk = scipy.ndimage.binary_dilation(msk, iterations=7)
pl.imshow(msk, interpolation='none', origin='lower')
pl.title('mask')
pl.tight_layout()
pl.figure()

from dask_image.ndmorph import binary_dilation, binary_erosion
m3d = (scube > 25*u.K).include()
m3d = binary_erosion(m3d, iterations=1)
m3d = binary_dilation(m3d, iterations=2)
m3d = m3d.compute()

m1 = scube.with_mask(msk).with_mask(m3d).moment1()
m0m = scube.with_mask(msk).with_mask(m3d).moment0()
pl.figure(figsize=(14,6))
ax1 = pl.subplot(1,2,1)
im = ax1.imshow(m1.value, vmin=vcen2.value-15, vmax=vcen2.value+15)
pl.colorbar(mappable=im)
mxv = scube.with_mask(msk).argmax_world(axis=0)
pl.subplot(1,2,2)
pl.imshow(mxv.value, vmin=vcen2.value-15, vmax=vcen2.value+15)
pl.colorbar()


pl.figure()
pl.imshow(m0m.value)

vmap2 = m1.hdu

In [ ]:
import sys
sys.path.append('/orange/adamginsburg/salt/code/')
import imp, overplot_saltlines; imp.reload(overplot_saltlines)
from overplot_saltlines import overplot_saltlines

In [ ]:
import imp
import overplot_saltlines
imp.reload(overplot_saltlines)
import lines
imp.reload(lines)
from overplot_saltlines import overplot_saltlines, all_lines
'SO65-54' in all_lines

In [ ]:
spectra = []
for ii,cube in enumerate(cubes):
    cube.allow_huge_operations=True
    sp = cube.subcube_from_regions(disk1).with_spectral_unit(u.GHz).mean(axis=(1,2))
    pysp = pyspeckit.Spectrum(data=sp, xarr=sp.spectral_axis, header=sp.header)
    spectra.append(pysp)
    pysp.specname = f'disk1_spw{ii+1}'

In [ ]:
spectra2 = []
for ii,cube in enumerate(cubesd2):
    cube.allow_huge_operations=True
    sp = cube.subcube_from_regions(disk2).with_spectral_unit(u.GHz).mean(axis=(1,2))
    pysp = pyspeckit.Spectrum(data=sp, xarr=sp.spectral_axis, header=sp.header)
    spectra2.append(pysp)
    pysp.specname = f'disk2_spw{ii+1}'

In [ ]:
mkdir -p spectra/figures/disk1

In [ ]:
mkdir -p spectra/figures/disk2

In [ ]:
overplot_saltlines(spectra, vcen=vcen, savepath='spectra/figures/disk1', ymax=180, ymin=-25)

In [ ]:
overplot_saltlines(spectra2, vcen=vcen, savepath='spectra/figures/disk2', ymax=180, ymin=-25)

In [ ]:
from IPython.display import Image,display

In [ ]:
for fn in glob.glob('spectra/figures/disk1/color*png'):
    display(Image(fn))

In [ ]:
for fn in glob.glob('spectra/figures/disk2/color*png'):
    display(Image(fn))

In [ ]:
import glob
import spectral_cube.analysis_utilities
import reproject
from tqdm.notebook import tqdm

for reg, cubelist, basename, vm in zip((disk1, disk2), (cubes, cubesd2), ('disk1', 'disk2'), (vmap, vmap2)):
    print(basename)
    for spwn,cube in enumerate(cubelist):
        print(cube)
        scube = cube.subcube_from_regions(reg).to(u.K)
        # continuum subtract before stacking
        #scube = scube - scube.percentile(25, axis=0)
        #mx = scube.max(axis=0)
        #msk = mx > 0.003*mx.unit
        print("Mean spectrum")
        sp = scube.mean(axis=(1,2))

        #sp.write(f'spectra/{fn.split("/")[-1].replace(".fits","_meanspec.fits")}', overwrite=True)


        # reproject the velocity map into the cube's coordinate system
        vmap_proj,_ = reproject.reproject_interp(vm,
                                                 scube.wcs.celestial,
                                                 shape_out=scube.shape[1:])
        vmap_proj = u.Quantity(vmap_proj, u.km/u.s)

        # convert to km/s with arbitrary rest value so 0 km/s can be interpreted
        if scube.spectral_axis.unit.is_equivalent(u.Hz):
            scube = scube.with_spectral_unit(u.km/u.s, velocity_convention='radio', rest_value=scube.spectral_axis.mean())

        print("Stacking")
        # perform the stacking!
        stack = spectral_cube.analysis_utilities.stack_spectra(scube, vmap_proj,
                                                               v0=0.0*u.km/u.s,
                                                               #progressbar=tqdm,
                                                               #num_cores=8
                                                              )
        fstack = stack.with_spectral_unit(u.GHz)

        pl.suptitle(f"{fn}")
        fstack.write(f'spectra/{basename}_{spwn}_stack.fits', overwrite=True)

        pl.clf()
        fstack.quicklook(filename=f'spectra/figures/{basename}_{spwn}_stack.pdf')
        fstack.quicklook(filename=f'spectra/figures/{basename}_{spwn}_stack.png')


        fig = pl.figure()
        mappable = pl.subplot(1,2,1).imshow(mx.value, origin='lower', interpolation='none')
        pl.colorbar(mappable)
        pl.subplot(1,2,2)
        pl.suptitle(fn)
        sp.quicklook()

In [ ]:
ls spectra

In [ ]:
stackspectra = []
for ii,fn in enumerate(glob.glob('spectra/*_stack.fits')):
    pysp = pyspeckit.Spectrum(fn)
    basename = os.path.basename(fn)
    spl = basename.split("_")
    name = 'mm1' if spl[0] == disk1 else 'mm2'
    pysp.specname = f'{name}_{spl[1]}_stack'
    stackspectra.append(pysp)

In [ ]:
# changed from vcen=vcen to vcen=0 b/c these are stacked - TODO check this?
overplot_saltlines(stackspectra, vcen=0*u.km/u.s, savepath='spectra/figures', ymax=200, ymin=-25)

rm spectra/figures/color_labels_G351_image_co1_smaller_stack_stack.png

rm spectra/figures/color_labels_G351_54_stack.png

In [ ]:
ls spectra/figures/

In [ ]:
for fn in glob.glob('spectra/figures/color*stack.png'):
    display(Image(fn))

# Try to ID the lines from Beuther+ that were only tentative

In [ ]:
from astroquery.splatalogue import Splatalogue

In [ ]:
sp = stackspectra[6]
lx,ux = 234.75,234.9
sp.plotter(xmin=lx, xmax=ux)
sp.baseline()
sp.plotter(xmin=lx, xmax=ux)
sp.specfit() #guesses=[5,216.935,0.01], limited=[(True,True),(True,True),(True,True)], limits=[(0,20),(216.93,216.94),(0.0001,0.05)])
sp.baseline(excludefit=True)
sp.plotter(xmin=lx, xmax=ux, ymax=150)
sp.specfit()
cen = sp.specfit.parinfo['SHIFT0']*sp.xarr.unit #* (1 + vcen/constants.c)
dx = (5*u.km/u.s / constants.c)*cen
Splatalogue.query_lines(cen-dx, cen+dx, line_lists=['CDMS'], energy_max=10000, energy_type='eu_k', chemical_name="monosulfide")


In [ ]:
sp = stackspectra[6]
lx,ux = 234.9,235
sp.plotter(xmin=lx, xmax=ux)
sp.baseline()
sp.plotter(xmin=lx, xmax=ux)
sp.specfit() #guesses=[5,216.935,0.01], limited=[(True,True),(True,True),(True,True)], limits=[(0,20),(216.93,216.94),(0.0001,0.05)])
sp.baseline(excludefit=True)
sp.plotter(xmin=lx, xmax=ux, ymax=150)
sp.specfit()
cen = sp.specfit.parinfo['SHIFT0']*sp.xarr.unit #* (1 + vcen/constants.c)
dx = (5*u.km/u.s / constants.c)*cen
tbl = Splatalogue.query_lines(cen-dx, cen+dx, energy_max=10000, energy_type='eu_k', chemical_name='PN')
tbl.show_in_notebook(display_length=150)
#cen

In [ ]:
sp = stackspectra[6]
lx,ux = 235.9,236.05
sp.plotter(xmin=lx, xmax=ux)
sp.baseline()
sp.plotter(xmin=lx, xmax=ux)
sp.specfit() #guesses=[5,216.935,0.01], limited=[(True,True),(True,True),(True,True)], limits=[(0,20),(216.93,216.94),(0.0001,0.05)])
sp.baseline(excludefit=True)
sp.plotter(xmin=lx, xmax=ux, ymax=150)
sp.specfit()
cen = sp.specfit.parinfo['SHIFT0']*sp.xarr.unit #* (1 + vcen/constants.c)
dx = (5*u.km/u.s / constants.c)*cen
tbl = Splatalogue.query_lines(cen-dx, cen+dx, energy_max=10000, energy_type='eu_k', chemical_name='SiS')
tbl.show_in_notebook(display_length=150)

In [ ]:
Splatalogue.query_lines(215*u.GHz, 236*u.GHz, energy_max=10000, energy_type='eu_k', chemical_name='Silicon monosulfide')

In [ ]:
Splatalogue.query_lines(215*u.GHz, 236*u.GHz, energy_max=10000, energy_type='eu_k', chemical_name='PN')